In [1]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import stats

# Preparing data set for Tableau

In [ ]:
subcribers = pd.read_csv("data/master_subscribers.csv")
subcribers.info()

In [ ]:
df1 = pd.read_csv("data/master_cases_clients.csv")
df1.head()

In [ ]:
df1.info()

In [ ]:
df = pd.merge(subcribers, df1, how='left', on='lab_name')
df.info()

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['case_id'], inplace=True)
df.info()

In [ ]:
df.drop('bClientLab', axis=1, inplace=True)
df.drop('master_pricebook_id', axis=1, inplace=True)
df.drop('custom_pricebook_id', axis=1, inplace=True)

In [ ]:
df2 = pd.read_csv("data/master_case_items.csv")
df2.info()

In [ ]:
df2.drop_duplicates(keep='first', inplace=True)
df2.info()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.drop('product_id', axis=1, inplace=True)

In [ ]:
df = pd.merge(df, df2, how='left', on=['lab_name', 'case_id'])
df.info()

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['case_id'] = df['case_id'].astype(np.int64)

In [ ]:
df3 = pd.read_csv("data/master_units_billing.csv")
df3.drop('Unnamed: 0', axis=1, inplace=True)
df3.info()

In [ ]:
df3.drop_duplicates(keep='first', inplace=True)
df3.info()

In [ ]:
df = pd.merge(df, df3, how='left', on=['lab_name', 'case_id', 'unit_id'])
df.info()

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
#df.info()

In [ ]:
df[['case_item_product_id', 'product_id', 'case_item_tooth_num', 'tooth_num']].tail(50) # drop 'product_id' and 'tooth_num'


In [ ]:
df.drop('product_id', axis=1, inplace=True)
df.drop('tooth_num', axis=1, inplace=True)
df.rename(columns={'case_item_product_id': 'unit_product_id', 'case_item_tooth_num': 'unit_tooth_num',
                  'case_item_crown_type': 'unit_crown_type', 'case_item_created_on': 'unit_created_on',
                  'tooth_type_oid': 'unit_tooth_type_oid', 'case_item_last_updated': 'unit_last_updated',
                  'case_item_nb_other_settings': 'unit_nb_other_settings', 'product_name': 'unit_product_name',
                  'product_type_id': 'unit_product_type_id', 'product_type_name': 'unit_product_type_name',
                  'tooth_type_label': 'unit_tooth_type_label'}, inplace=True)
df.info()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Drop rows where 'case_created_on' is null
df.dropna(subset=['case_created_on'], inplace=True)

In [ ]:
# Convert 'case_created_on' to datetime
df['case_created_on'] = pd.to_datetime(df['case_created_on'])

In [ ]:
# Drop rows where 'case_created_on' is before 1/1/2011
df = df[df.case_created_on >= datetime(2011, 1, 1)]
df.info()

In [ ]:
df.client_name.unique()

In [ ]:
df = df[df.client_name != 'Test Client']
df.info()

In [ ]:
# Save to csv
df.to_csv('data/labstar_tableau.csv', index=False, header=True)

# Preparing data set for cluster analysis

In [ ]:
df = pd.read_csv("data/labstar_tableau.csv")

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
non_us = pd.read_csv("data/currency.csv")
non_us

In [ ]:
non_us.ID

In [ ]:
# Separate subscribers using USD vs another currency
other = df[df.subscriber_id.isin(non_us.ID)]
usa = df[~df.subscriber_id.isin(non_us.ID)]
print(usa.info())
print(other.info())

In [ ]:
usa[['billing_item_amount', 'charging_item_quantity']].describe()

In [ ]:
usa[['billing_item_amount', 'charging_item_quantity']].quantile(.99)

In [ ]:
usa.info()

In [ ]:
usa.isnull().sum()

In [ ]:
usa.drop_duplicates(keep='first', inplace=True)

In [ ]:
usa.info()

In [ ]:
df_usa = usa[['lab_name', 'subscriber_plan_id', 'subscriber_plan_name', 'nb_subscriber_features',
              'case_id', 'unit_id', 'case_state', 'case_created_on', 'unit_last_updated',
              'billing_item_created_on', 'unit_tooth_num', 'unit_tooth_type_oid', 'unit_tooth_type_label',
              'unit_crown_type', 'unit_product_id', 'unit_product_name', 'unit_product_type_id',
              'unit_product_type_name', 'unit_nb_other_settings', 'case_nb_other_products', 'client_id',
              'client_name', 'shipping_zipcode', 'case_client_estimated_price', 'unit_client_estimated_price',
              'billing_item_amount', 'billing_item_payment', 'billing_item_status']]

In [ ]:
df_usa.to_csv('data/df_usa.csv', index=False)

### Preliminary prep steps

- Transform dtype where appropriate
- Create columns 'month' and 'day' for grouping purposes down the road
- Calculate time elapsed in days between 'case_created_on' and 5/1/2019 ('days_subscriber')
- Fill NaN with placeholder -9999999 in 'case_state', 'pt_id', 'pto_id', 'product_additional_id', 'client_estimated_price', 'billing_item_amount', 'billing_item_payment', 'billing_item_status', and 'days_to_billing'
- Calculate diff between 'client_estimated_price' and 'billing_item_amount'
- Include field to indicate if billed amount was over the client_estimated_price (1 if true, else 0)
- Calculate diff between 'billing_item_amount' and 'billing_item_payment'
- Include field to indicate if paid amount was less than the billed amount (1 if true, else 0)
 

In [2]:
df = pd.read_csv('data/DATA_TEMP.csv')

In [10]:
df.drop_duplicates(keep='first', inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9467209 entries, 0 to 9467208
Data columns (total 13 columns):
lab_name                  object
subscriber_plan_name      object
nb_subscriber_features    int64
case_id                   int64
unit_id                   float64
case_created_on           object
unit_tooth_num            float64
unit_product_id           float64
unit_product_type_id      float64
unit_nb_other_settings    float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
dtypes: float64(7), int64(2), object(4)
memory usage: 1011.2+ MB


In [8]:
df.isnull().sum()

lab_name                        0
subscriber_plan_name            0
nb_subscriber_features          0
case_id                         0
unit_id                     52425
case_created_on                 0
unit_tooth_num              52425
unit_product_id             52425
unit_product_type_id        53527
unit_nb_other_settings    8852939
case_nb_other_products    8999974
client_id                      35
shipping_zipcode           997496
dtype: int64

In [7]:
df.drop('month', axis=1, inplace=True)
df.drop('day', axis=1, inplace=True)

In [12]:
# Transform dtype where appropriate
df['case_created_on'] = pd.to_datetime(df['case_created_on'])
#df_usa['case_created_on'] = pd.to_datetime(df_usa['case_created_on'])
#df_usa['unit_last_updated'] = pd.to_datetime(df_usa['unit_last_updated'])
#df_usa['billing_item_created_on'] = pd.to_datetime(df_usa['billing_item_created_on'])

In [13]:
# Create column 'month'
df['month'] = df['case_created_on'].apply(lambda x: x.strftime('%Y-%m'))
df[['case_created_on', 'month']].head()

,case_created_on,month
0,2017-06-28 18:48:08.793,2017-06
1,2017-06-28 18:48:08.793,2017-06
2,2017-06-28 18:48:08.793,2017-06
3,2017-06-29 12:43:38.153,2017-06
4,2017-06-29 12:43:38.153,2017-06


In [14]:
# Create column 'day'
df['day'] = df['case_created_on'].apply(lambda x: x.strftime('%Y-%m-%d'))
df[['case_created_on', 'day']].head()

,case_created_on,day
0,2017-06-28 18:48:08.793,2017-06-28
1,2017-06-28 18:48:08.793,2017-06-28
2,2017-06-28 18:48:08.793,2017-06-28
3,2017-06-29 12:43:38.153,2017-06-29
4,2017-06-29 12:43:38.153,2017-06-29


In [15]:
df.day.max()

'2019-05-05'

In [16]:
# Calculate time elapsed in days between 'case_created_on' and 5/1/2019
df['days_subscribed'] = df['case_created_on'].apply(lambda x: (datetime(2019, 5, 5) - x)/np.timedelta64(1,'D'))
df['days_subscribed'] = df['days_subscribed'].astype(np.int64)
df[['case_created_on', 'days_subscribed']].head()


,case_created_on,days_subscribed
0,2017-06-28 18:48:08.793,675
1,2017-06-28 18:48:08.793,675
2,2017-06-28 18:48:08.793,675
3,2017-06-29 12:43:38.153,674
4,2017-06-29 12:43:38.153,674


In [17]:
df[['case_created_on', 'days_subscribed']].sort_values(by=['case_created_on'], ascending=False)

,case_created_on,days_subscribed
147994,2019-05-05 18:42:03.443,0
2721938,2019-05-05 18:06:13.480,0
2721937,2019-05-05 18:05:37.233,0
2721936,2019-05-05 18:05:00.077,0
2721935,2019-05-05 18:04:29.860,0
2721934,2019-05-05 18:03:56.190,0
2721933,2019-05-05 18:03:17.503,0
2721932,2019-05-05 18:02:35.253,0
2721931,2019-05-05 18:02:02.800,0
2721930,2019-05-05 18:01:27.833,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9467209 entries, 0 to 9467208
Data columns (total 16 columns):
lab_name                  object
subscriber_plan_name      object
nb_subscriber_features    int64
case_id                   int64
unit_id                   float64
case_created_on           datetime64[ns]
unit_tooth_num            float64
unit_product_id           float64
unit_product_type_id      float64
unit_nb_other_settings    float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
month                     object
day                       object
days_subscribed           int64
dtypes: datetime64[ns](1), float64(7), int64(3), object(5)
memory usage: 1.2+ GB


In [19]:
df.drop_duplicates(keep='first', inplace=True)

In [21]:
#Save to csv
df.to_csv('data/master_DATA_for_analysis.csv', index=False, header=True)

### Aggregation steps: aggregate by month, lab_name, case

- Group data by 'lab_name', 'month', 'day', 'case_id', 'unit_id'
- Group data by 'lab_name', 'month', 'day', 'case_id'
- Group data by 'lab_name', 'month', 'day'
- Group data by 'lab_name', 'month'
- Group data by 'lab_name'

In [22]:
data_set = df
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9467209 entries, 0 to 9467208
Data columns (total 16 columns):
lab_name                  object
subscriber_plan_name      object
nb_subscriber_features    int64
case_id                   int64
unit_id                   float64
case_created_on           datetime64[ns]
unit_tooth_num            float64
unit_product_id           float64
unit_product_type_id      float64
unit_nb_other_settings    float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
month                     object
day                       object
days_subscribed           int64
dtypes: datetime64[ns](1), float64(7), int64(3), object(5)
memory usage: 1.2+ GB


In [23]:
data_set.describe()

,nb_subscriber_features,case_id,unit_id,unit_tooth_num,unit_product_id,unit_product_type_id,unit_nb_other_settings,case_nb_other_products,client_id,days_subscribed
count,9.467209e+06,9.467209e+06,9.414784e+06,9.414784e+06,9.414784e+06,9.413682e+06,614270.000000,467235.000000,9.467174e+06,9.467209e+06
mean,1.991852e+01,2.240958e+04,4.495475e+04,3.106048e+01,1.153729e+02,1.718760e+01,1.271356,1.256259,3.793253e+02,6.683749e+02
std,4.452000e+00,2.625813e+04,5.440734e+04,3.222823e+01,7.263168e+01,2.006561e+01,0.607975,0.807327,1.450764e+03,5.241164e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,0.000000,-1.000000e+00,0.000000e+00
25%,1.900000e+01,4.708000e+03,9.124000e+03,9.000000e+00,8.800000e+01,1.200000e+01,1.000000,1.000000,1.900000e+01,2.470000e+02
50%,2.100000e+01,1.281300e+04,2.467600e+04,1.800000e+01,9.900000e+01,1.600000e+01,1.000000,1.000000,6.400000e+01,5.480000e+02
75%,2.300000e+01,3.052300e+04,6.087500e+04,3.100000e+01,1.360000e+02,2.100000e+01,1.000000,1.000000,1.910000e+02,9.830000e+02
max,2.300000e+01,1.708730e+05,3.787790e+05,9.200000e+01,1.198000e+03,1.038000e+03,23.000000,54.000000,1.320100e+04,2.865000e+03


In [24]:
data_set.isnull().sum()

lab_name                        0
subscriber_plan_name            0
nb_subscriber_features          0
case_id                         0
unit_id                     52425
case_created_on                 0
unit_tooth_num              52425
unit_product_id             52425
unit_product_type_id        53527
unit_nb_other_settings    8852939
case_nb_other_products    8999974
client_id                      35
shipping_zipcode           997496
month                           0
day                             0
days_subscribed                 0
dtype: int64

In [25]:
data = data_set.dropna(subset=['unit_id'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9414784 entries, 0 to 9467208
Data columns (total 16 columns):
lab_name                  object
subscriber_plan_name      object
nb_subscriber_features    int64
case_id                   int64
unit_id                   float64
case_created_on           datetime64[ns]
unit_tooth_num            float64
unit_product_id           float64
unit_product_type_id      float64
unit_nb_other_settings    float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
month                     object
day                       object
days_subscribed           int64
dtypes: datetime64[ns](1), float64(7), int64(3), object(5)
memory usage: 1.2+ GB


In [26]:
data.drop_duplicates(keep='first', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9414784 entries, 0 to 9467208
Data columns (total 16 columns):
lab_name                  object
subscriber_plan_name      object
nb_subscriber_features    int64
case_id                   int64
unit_id                   float64
case_created_on           datetime64[ns]
unit_tooth_num            float64
unit_product_id           float64
unit_product_type_id      float64
unit_nb_other_settings    float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
month                     object
day                       object
days_subscribed           int64
dtypes: datetime64[ns](1), float64(7), int64(3), object(5)
memory usage: 1.2+ GB


In [27]:
data.columns

Index(['lab_name', 'subscriber_plan_name', 'nb_subscriber_features', 'case_id',
       'unit_id', 'case_created_on', 'unit_tooth_num', 'unit_product_id',
       'unit_product_type_id', 'unit_nb_other_settings',
       'case_nb_other_products', 'client_id', 'shipping_zipcode', 'month',
       'day', 'days_subscribed'],
      dtype='object')

In [30]:
# Set aggregations for groupby by 'lab_name', 'month', 'day', 'case_id', 'unit_id'
agg_by_all = {
    'subscriber_plan_name': 'first',
    'nb_subscriber_features': 'first',
    'unit_tooth_num': 'first',
    'unit_product_id': 'count',
    'unit_product_type_id': 'count',
    'unit_nb_other_settings': 'sum',
    'case_nb_other_products': 'first',
    'client_id': 'first',
    'shipping_zipcode': 'first',
    'days_subscribed': 'first'   
}

In [31]:
# Group data_set by 'lab_name', 'month', 'day', 'case_id', 'unit_id'
g_by_all = data.groupby(by=['lab_name', 'month', 'day', 'case_id', 'unit_id'], as_index=False)
all_groups = g_by_all.agg(agg_by_all)
all_groups.head()

,lab_name,month,day,case_id,unit_id,subscriber_plan_name,nb_subscriber_features,unit_tooth_num,unit_product_id,unit_product_type_id,unit_nb_other_settings,case_nb_other_products,client_id,shipping_zipcode,days_subscribed
0,3DDENTALLABORATORIES,2017-06,2017-06-28,5,8.0,Standard,16,12.0,1,1,0.0,2.0,10.0,55420,675
1,3DDENTALLABORATORIES,2017-06,2017-06-28,5,9.0,Standard,16,13.0,1,1,0.0,2.0,10.0,55420,675
2,3DDENTALLABORATORIES,2017-06,2017-06-28,5,10.0,Standard,16,14.0,1,1,0.0,2.0,10.0,55420,675
3,3DDENTALLABORATORIES,2017-06,2017-06-29,7,15.0,Standard,16,13.0,1,1,0.0,NaN,10.0,55420,674
4,3DDENTALLABORATORIES,2017-06,2017-06-29,7,16.0,Standard,16,14.0,1,1,0.0,NaN,10.0,55420,674


In [32]:
all_groups.rename(columns={'unit_product_id': 'nb_products',
                           'unit_product_type_id': 'nb_product_types',
                           'unit_nb_other_settings': 'nb_other_settings'}, inplace=True)

In [35]:
all_groups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9414774 entries, 0 to 9414773
Data columns (total 15 columns):
lab_name                  object
month                     object
day                       object
case_id                   int64
unit_id                   float64
subscriber_plan_name      object
nb_subscriber_features    int64
unit_tooth_num            float64
nb_products               int64
nb_product_types          int64
nb_other_settings         float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
days_subscribed           int64
dtypes: float64(5), int64(5), object(5)
memory usage: 1.1+ GB


In [34]:
all_groups.drop_duplicates(keep='first', inplace=True)

In [36]:
# Hot encode tooth_num
tooth_num = all_groups['unit_tooth_num']
tooth_num = pd.get_dummies(tooth_num)
print(tooth_num.head())
tooth_num.columns = tooth_num.columns.astype(int).astype(str)
for n in tooth_num.columns:
    tooth_num.rename(columns={n: 'tooth_' + n}, inplace=True)
print(tooth_num.columns)
all_groups = pd.concat([all_groups, tooth_num], axis=1)
all_groups.head()

   0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0   9.0   ...   26.0  \
0     0     0     0     0     0     0     0     0     0     0  ...      0   
1     0     0     0     0     0     0     0     0     0     0  ...      0   
2     0     0     0     0     0     0     0     0     0     0  ...      0   
3     0     0     0     0     0     0     0     0     0     0  ...      0   
4     0     0     0     0     0     0     0     0     0     0  ...      0   

   27.0  28.0  29.0  30.0  31.0  32.0  90.0  91.0  92.0  
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 36 columns]
Index(['tooth_0', 'tooth_1', 'tooth_2', 'tooth_3', 'tooth_4', 'tooth_5',
       'tooth_6', 'tooth_7', 'tooth_8', 'tooth_9', 'tooth_10', 'tooth_11',
       'tooth_12',

,lab_name,month,day,case_id,unit_id,subscriber_plan_name,nb_subscriber_features,unit_tooth_num,nb_products,nb_product_types,...,tooth_26,tooth_27,tooth_28,tooth_29,tooth_30,tooth_31,tooth_32,tooth_90,tooth_91,tooth_92
0,3DDENTALLABORATORIES,2017-06,2017-06-28,5,8.0,Standard,16,12.0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,3DDENTALLABORATORIES,2017-06,2017-06-28,5,9.0,Standard,16,13.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,3DDENTALLABORATORIES,2017-06,2017-06-28,5,10.0,Standard,16,14.0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,3DDENTALLABORATORIES,2017-06,2017-06-29,7,15.0,Standard,16,13.0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,3DDENTALLABORATORIES,2017-06,2017-06-29,7,16.0,Standard,16,14.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [41]:
all_groups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9414774 entries, 0 to 9414773
Data columns (total 51 columns):
lab_name                  object
month                     object
day                       object
case_id                   int64
unit_id                   float64
subscriber_plan_name      object
nb_subscriber_features    int64
unit_tooth_num            float64
nb_products               int64
nb_product_types          int64
nb_other_settings         float64
case_nb_other_products    float64
client_id                 float64
shipping_zipcode          object
days_subscribed           int64
tooth_0                   uint8
tooth_1                   uint8
tooth_2                   uint8
tooth_3                   uint8
tooth_4                   uint8
tooth_5                   uint8
tooth_6                   uint8
tooth_7                   uint8
tooth_8                   uint8
tooth_9                   uint8
tooth_10                  uint8
tooth_11                  uint8
tooth_12

In [39]:
all_groups.drop_duplicates(keep='first', inplace=True)

In [40]:
all_groups.columns

Index(['lab_name', 'month', 'day', 'case_id', 'unit_id',
       'subscriber_plan_name', 'nb_subscriber_features', 'unit_tooth_num',
       'nb_products', 'nb_product_types', 'nb_other_settings',
       'case_nb_other_products', 'client_id', 'shipping_zipcode',
       'days_subscribed', 'tooth_0', 'tooth_1', 'tooth_2', 'tooth_3',
       'tooth_4', 'tooth_5', 'tooth_6', 'tooth_7', 'tooth_8', 'tooth_9',
       'tooth_10', 'tooth_11', 'tooth_12', 'tooth_13', 'tooth_14', 'tooth_15',
       'tooth_16', 'tooth_17', 'tooth_18', 'tooth_19', 'tooth_20', 'tooth_21',
       'tooth_22', 'tooth_23', 'tooth_24', 'tooth_25', 'tooth_26', 'tooth_27',
       'tooth_28', 'tooth_29', 'tooth_30', 'tooth_31', 'tooth_32', 'tooth_90',
       'tooth_91', 'tooth_92'],
      dtype='object')

In [42]:
# Set aggregations for groupby by 'lab_name', 'month', 'day', 'case_id'
agg_by_case = {
    'subscriber_plan_name': {'subscriber_plan_name': 'first'},
    'nb_subscriber_features': {'nb_subscriber_features': 'first'},
    'unit_id': {'nb_units': 'count'},
    'nb_products': {'nb_products_per_case': 'sum',
                    'avg_nb_products_per_unit': 'mean'},
    'nb_product_types': {'nb_product_types_per_case': 'sum',
                        'avg_nb_product_types_per_unit': 'mean'},
    'nb_other_settings': {'nb_other_settings_per_case': 'sum',
                         'avg_nb_other_settings_per_unit': 'mean'},
    'case_nb_other_products': {'case_nb_other_products': 'first'},
    'client_id': {'client_id': 'first'},
    'shipping_zipcode': {'shipping_zipcode': 'first'},
    'days_subscribed': {'days_subscribed': 'first'},
    'tooth_0': {'nb_tooth_0_per_case': 'sum'},
    'tooth_1': {'nb_tooth_1_per_case': 'sum'},
    'tooth_2': {'nb_tooth_2_per_case': 'sum'},
    'tooth_3': {'nb_tooth_3_per_case': 'sum'},
    'tooth_4': {'nb_tooth_4_per_case': 'sum'},
    'tooth_5': {'nb_tooth_5_per_case': 'sum'},
    'tooth_6': {'nb_tooth_6_per_case': 'sum'},
    'tooth_7': {'nb_tooth_7_per_case': 'sum'},
    'tooth_8': {'nb_tooth_8_per_case': 'sum'},
    'tooth_9': {'nb_tooth_9_per_case': 'sum'},
    'tooth_10': {'nb_tooth_10_per_case': 'sum'},
    'tooth_11': {'nb_tooth_11_per_case': 'sum'},
    'tooth_12': {'nb_tooth_12_per_case': 'sum'},
    'tooth_13': {'nb_tooth_13_per_case': 'sum'},
    'tooth_14': {'nb_tooth_14_per_case': 'sum'},
    'tooth_15': {'nb_tooth_15_per_case': 'sum'},
    'tooth_16': {'nb_tooth_16_per_case': 'sum'},
    'tooth_17': {'nb_tooth_17_per_case': 'sum'},
    'tooth_18': {'nb_tooth_18_per_case': 'sum'},
    'tooth_19': {'nb_tooth_19_per_case': 'sum'},
    'tooth_20': {'nb_tooth_20_per_case': 'sum'},
    'tooth_21': {'nb_tooth_21_per_case': 'sum'},
    'tooth_22': {'nb_tooth_22_per_case': 'sum'},
    'tooth_23': {'nb_tooth_23_per_case': 'sum'},
    'tooth_24': {'nb_tooth_24_per_case': 'sum'},
    'tooth_25': {'nb_tooth_25_per_case': 'sum'},
    'tooth_26': {'nb_tooth_26_per_case': 'sum'},
    'tooth_27': {'nb_tooth_27_per_case': 'sum'},
    'tooth_28': {'nb_tooth_28_per_case': 'sum'},
    'tooth_29': {'nb_tooth_29_per_case': 'sum'},
    'tooth_30': {'nb_tooth_30_per_case': 'sum'},
    'tooth_31': {'nb_tooth_31_per_case': 'sum'},
    'tooth_32': {'nb_tooth_32_per_case': 'sum'},
    'tooth_90': {'nb_tooth_90_per_case': 'sum'},
    'tooth_91': {'nb_tooth_91_per_case': 'sum'},
    'tooth_92': {'nb_tooth_92_per_case': 'sum'}
}

In [43]:
# Group data_set by 'lab_name', 'month', 'day', 'case_id'
g_by_case = all_groups.groupby(by=['lab_name', 'month', 'day', 'case_id'])
case_groups = g_by_case.agg(agg_by_case)
case_groups.head()

subscriber_plan_name  \
                                                subscriber_plan_name   
lab_name             month   day        case_id                        
3DDENTALLABORATORIES 2017-06 2017-06-28 5                   Standard   
                             2017-06-29 7                   Standard   
                                        8                   Standard   
                             2017-06-30 9                   Standard   
                                        11                  Standard   

                                                nb_subscriber_features  \
                                                nb_subscriber_features   
lab_name             month   day        case_id                          
3DDENTALLABORATORIES 2017-06 2017-06-28 5                           16   
                             2017-06-29 7                           16   
                                        8                           16   
                             2017-06-30 9                           16   
                                        11                          16   

                                                 unit_id          nb_products  \
                                                nb_units nb_products_per_case   
lab_name             month   day        case_id                                 
3DDENTALLABORATORIES 2017-06 2017-06-28 5              3                    3   
                             2017-06-29 7              3                    3   
                                        8              1                    1   
                             2017-06-30 9              3                    3   
                                        11             1                    1   

                                                                          \
                                                avg_nb_products_per_unit   
lab_name             month   day        case_id                            
3DDENTALLABORATORIES 2017-06 2017-06-28 5                            1.0   
                             2017-06-29 7                            1.0   
                                        8                            1.0   
                             2017-06-30 9                            1.0   
                                        11                           1.0   

                                                         nb_product_types  \
                                                nb_product_types_per_case   
lab_name             month   day        case_id                             
3DDENTALLABORATORIES 2017-06 2017-06-28 5                               3   
                             2017-06-29 7                               3   
                                        8                               1   
                             2017-06-30 9                               3   
                                        11                              1   

                                                                               \
                                                avg_nb_product_types_per_unit   
lab_name             month   day        case_id                                 
3DDENTALLABORATORIES 2017-06 2017-06-28 5                                 1.0   
                             2017-06-29 7                                 1.0   
                                        8                                 1.0   
                             2017-06-30 9                                 1.0   
                                        11                                1.0   

                                                         nb_other_settings  \
                                                nb_other_settings_per_case   
lab_name             month   day        case_id                              
3DDENTALLABORATORIES 2017-06 2017-06-28 5                              0.0   
                             2017-06-29 7 

In [44]:
case_groups.columns = case_groups.columns.droplevel(level=0)
case_groups.head()

subscriber_plan_name  \
lab_name             month   day        case_id                        
3DDENTALLABORATORIES 2017-06 2017-06-28 5                   Standard   
                             2017-06-29 7                   Standard   
                                        8                   Standard   
                             2017-06-30 9                   Standard   
                                        11                  Standard   

                                                 nb_subscriber_features  \
lab_name             month   day        case_id                           
3DDENTALLABORATORIES 2017-06 2017-06-28 5                            16   
                             2017-06-29 7                            16   
                                        8                            16   
                             2017-06-30 9                            16   
                                        11                           16   

                                                 nb_units  \
lab_name             month   day        case_id             
3DDENTALLABORATORIES 2017-06 2017-06-28 5               3   
                             2017-06-29 7               3   
                                        8               1   
                             2017-06-30 9               3   
                                        11              1   

                                                 nb_products_per_case  \
lab_name             month   day        case_id                         
3DDENTALLABORATORIES 2017-06 2017-06-28 5                           3   
                             2017-06-29 7                           3   
                                        8                           1   
                             2017-06-30 9                           3   
                                        11                          1   

                                                 avg_nb_products_per_unit  \
lab_name             month   day        case_id                             
3DDENTALLABORATORIES 2017-06 2017-06-28 5                             1.0   
                             2017-06-29 7                             1.0   
                                        8                             1.0   
                             2017-06-30 9                             1.0   
                                        11                            1.0   

                                                 nb_product_types_per_case  \
lab_name             month   day        case_id                              
3DDENTALLABORATORIES 2017-06 2017-06-28 5                                3   
                             2017-06-29 7                                3   
                                        8                                1   
                             2017-06-30 9                                3   
                                        11                               1   

                                                 avg_nb_product_types_per_unit  \
lab_name             month   day        case_id                                  
3DDENTALLABORATORIES 2017-06 2017-06-28 5                                  1.0   
                             2017-06-29 7                                  1.0   
                                        8                                  1.0   
                             2017-06-30 9                                  1.0   
                                        11                                 1.0   

                                                 nb_other_settings_per_case  \
lab_name             month   day        case_id                               
3DDENTALLABORATORIES 2017-06 2017-06-28 5                               0.0   
                             2017-06-29 7                               0.0   
                                        8                               0.0   
                    

In [45]:
case_groups.reset_index(inplace=True)
case_groups.head()

,lab_name,month,day,case_id,subscriber_plan_name,nb_subscriber_features,nb_units,nb_products_per_case,avg_nb_products_per_unit,nb_product_types_per_case,...,nb_tooth_26_per_case,nb_tooth_27_per_case,nb_tooth_28_per_case,nb_tooth_29_per_case,nb_tooth_30_per_case,nb_tooth_31_per_case,nb_tooth_32_per_case,nb_tooth_90_per_case,nb_tooth_91_per_case,nb_tooth_92_per_case
0,3DDENTALLABORATORIES,2017-06,2017-06-28,5,Standard,16,3,3,1.0,3,...,0,0,0,0,0,0,0,0,0,0
1,3DDENTALLABORATORIES,2017-06,2017-06-29,7,Standard,16,3,3,1.0,3,...,0,0,0,0,0,0,0,0,0,0
2,3DDENTALLABORATORIES,2017-06,2017-06-29,8,Standard,16,1,1,1.0,1,...,0,0,0,0,0,0,0,0,0,0
3,3DDENTALLABORATORIES,2017-06,2017-06-30,9,Standard,16,3,3,1.0,3,...,0,0,0,0,0,0,0,0,0,0
4,3DDENTALLABORATORIES,2017-06,2017-06-30,11,Standard,16,1,1,1.0,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
case_groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5904296 entries, 0 to 5904295
Data columns (total 53 columns):
lab_name                          object
month                             object
day                               object
case_id                           int64
subscriber_plan_name              object
nb_subscriber_features            int64
nb_units                          int64
nb_products_per_case              int64
avg_nb_products_per_unit          float64
nb_product_types_per_case         int64
avg_nb_product_types_per_unit     float64
nb_other_settings_per_case        float64
avg_nb_other_settings_per_unit    float64
case_nb_other_products            float64
client_id                         float64
shipping_zipcode                  object
days_subscribed                   int64
nb_tooth_0_per_case               uint8
nb_tooth_1_per_case               uint8
nb_tooth_2_per_case               uint8
nb_tooth_3_per_case               uint8
nb_tooth_4_per_case           

In [ ]:
case_groups.drop_duplicates(keep='first', inplace=True)

In [47]:
case_groups.columns

Index(['lab_name', 'month', 'day', 'case_id', 'subscriber_plan_name',
       'nb_subscriber_features', 'nb_units', 'nb_products_per_case',
       'avg_nb_products_per_unit', 'nb_product_types_per_case',
       'avg_nb_product_types_per_unit', 'nb_other_settings_per_case',
       'avg_nb_other_settings_per_unit', 'case_nb_other_products', 'client_id',
       'shipping_zipcode', 'days_subscribed', 'nb_tooth_0_per_case',
       'nb_tooth_1_per_case', 'nb_tooth_2_per_case', 'nb_tooth_3_per_case',
       'nb_tooth_4_per_case', 'nb_tooth_5_per_case', 'nb_tooth_6_per_case',
       'nb_tooth_7_per_case', 'nb_tooth_8_per_case', 'nb_tooth_9_per_case',
       'nb_tooth_10_per_case', 'nb_tooth_11_per_case', 'nb_tooth_12_per_case',
       'nb_tooth_13_per_case', 'nb_tooth_14_per_case', 'nb_tooth_15_per_case',
       'nb_tooth_16_per_case', 'nb_tooth_17_per_case', 'nb_tooth_18_per_case',
       'nb_tooth_19_per_case', 'nb_tooth_20_per_case', 'nb_tooth_21_per_case',
       'nb_tooth_22_per_case', 'nb

In [50]:
# Set aggregations for groupby by 'lab_name', 'month', 'day'
agg_by_day = {
    'subscriber_plan_name': {'subscriber_plan_name': 'first'},
    'nb_subscriber_features': {'nb_subscriber_features': 'first'},
    'case_id': {'nb_cases_per_day': 'count'},
    'nb_units': {'nb_units_per_day': 'sum',
                'avg_nb_units_per_case': 'mean'},
    'nb_products_per_case': {'nb_products_per_day': 'sum',
                             'avg_nb_products_per_case': 'mean'},
    'avg_nb_products_per_unit': {'avg_nb_products_per_unit': 'mean'},
    'nb_product_types_per_case': {'nb_product_types_per_day': 'sum',
                                  'avg_nb_product_types_per_case': 'mean'},
    'avg_nb_product_types_per_unit': {'avg_nb_product_types_per_unit': 'mean'},    
    'nb_other_settings_per_case': {'total_nb_other_settings_per_day': 'sum',
                         'avg_nb_other_settings_per_case': 'mean'},
    'avg_nb_other_settings_per_unit': {'avg_nb_other_settings_per_unit': 'mean'},
    'case_nb_other_products': {'nb_other_products_per_day': 'sum',
                               'avg_nb_other_products_per_case': 'mean'},
    'client_id': {'nb_clients_per_day': 'count'},
    'shipping_zipcode': {'nb_shipping_zipcodes_per_day': 'count'},
    'days_subscribed': {'days_subscribed': 'first'},
    'nb_tooth_0_per_case': {'total_nb_tooth_0_per_day': 'sum',
                           'avg_nb_tooth_0_per_case': 'mean'},
    'nb_tooth_1_per_case': {'total_nb_tooth_1_per_day': 'sum',
                           'avg_nb_tooth_1_per_case': 'mean'},
    'nb_tooth_2_per_case': {'total_nb_tooth_2_per_day': 'sum',
                           'avg_nb_tooth_2_per_case': 'mean'},
    'nb_tooth_3_per_case': {'total_nb_tooth_3_per_day': 'sum',
                           'avg_nb_tooth_3_per_case': 'mean'},
    'nb_tooth_4_per_case': {'total_nb_tooth_4_per_day': 'sum',
                           'avg_nb_tooth_4_per_case': 'mean'},
    'nb_tooth_5_per_case': {'total_nb_tooth_5_per_day': 'sum',
                           'avg_nb_tooth_5_per_case': 'mean'},
    'nb_tooth_6_per_case': {'total_nb_tooth_6_per_day': 'sum',
                           'avg_nb_tooth_6_per_case': 'mean'},
    'nb_tooth_7_per_case': {'total_nb_tooth_7_per_day': 'sum',
                           'avg_nb_tooth_7_per_case': 'mean'},
    'nb_tooth_8_per_case': {'total_nb_tooth_8_per_day': 'sum',
                           'avg_nb_tooth_8_per_case': 'mean'},
    'nb_tooth_9_per_case': {'total_nb_tooth_9_per_day': 'sum',
                           'avg_nb_tooth_9_per_case': 'mean'},
    'nb_tooth_10_per_case': {'total_nb_tooth_10_per_day': 'sum',
                           'avg_nb_tooth_10_per_case': 'mean'},
    'nb_tooth_11_per_case': {'total_nb_tooth_11_per_day': 'sum',
                           'avg_nb_tooth_11_per_case': 'mean'},
    'nb_tooth_12_per_case': {'total_nb_tooth_12_per_day': 'sum',
                           'avg_nb_tooth_12_per_case': 'mean'},
    'nb_tooth_13_per_case': {'total_nb_tooth_13_per_day': 'sum',
                           'avg_nb_tooth_13_per_case': 'mean'},
    'nb_tooth_14_per_case': {'total_nb_tooth_14_per_day': 'sum',
                           'avg_nb_tooth_14_per_case': 'mean'},
    'nb_tooth_15_per_case': {'total_nb_tooth_15_per_day': 'sum',
                           'avg_nb_tooth_15_per_case': 'mean'},
    'nb_tooth_16_per_case': {'total_nb_tooth_16_per_day': 'sum',
                           'avg_nb_tooth_16_per_case': 'mean'},
    'nb_tooth_17_per_case': {'total_nb_tooth_17_per_day': 'sum',
                           'avg_nb_tooth_17_per_case': 'mean'},
    'nb_tooth_18_per_case': {'total_nb_tooth_18_per_day': 'sum',
                           'avg_nb_tooth_18_per_case': 'mean'},
    'nb_tooth_19_per_case': {'total_nb_tooth_19_per_day': 'sum',
                           'avg_nb_tooth_19_per_case': 'mean'},
    'nb_tooth_20_per_case': {'total_nb_tooth_20_per_day': 'sum',
                           'avg_nb_tooth_20_per_case': 'mean'},
    'nb_tooth_21_per_case': {'total_nb_tooth_21_per_day': 'sum',
                           'avg_nb_tooth_21_per_case': 'mean'},
    'nb_tooth_22_per_case': {'total_nb_tooth_22_per_day': 'sum',
                           'avg_nb_tooth_22_per_case': 'mean'},
    'nb_tooth_23_per_case': {'total_nb_tooth_23_per_day': 'sum',
                           'avg_nb_tooth_23_per_case': 'mean'},
    'nb_tooth_24_per_case': {'total_nb_tooth_24_per_day': 'sum',
                           'avg_nb_tooth_24_per_case': 'mean'},
    'nb_tooth_25_per_case': {'total_nb_tooth_25_per_day': 'sum',
                           'avg_nb_tooth_25_per_case': 'mean'},
    'nb_tooth_26_per_case': {'total_nb_tooth_26_per_day': 'sum',
                           'avg_nb_tooth_26_per_case': 'mean'},
    'nb_tooth_27_per_case': {'total_nb_tooth_27_per_day': 'sum',
                           'avg_nb_tooth_27_per_case': 'mean'},
    'nb_tooth_28_per_case': {'total_nb_tooth_28_per_day': 'sum',
                           'avg_nb_tooth_28_per_case': 'mean'},
    'nb_tooth_29_per_case': {'total_nb_tooth_29_per_day': 'sum',
                           'avg_nb_tooth_29_per_case': 'mean'},
    'nb_tooth_30_per_case': {'total_nb_tooth_30_per_day': 'sum',
                           'avg_nb_tooth_30_per_case': 'mean'},
    'nb_tooth_31_per_case': {'total_nb_tooth_31_per_day': 'sum',
                           'avg_nb_tooth_31_per_case': 'mean'},
    'nb_tooth_32_per_case': {'total_nb_tooth_32_per_day': 'sum',
                           'avg_nb_tooth_32_per_case': 'mean'},
    'nb_tooth_90_per_case': {'total_nb_tooth_90_per_day': 'sum',
                           'avg_nb_tooth_90_per_case': 'mean'},
    'nb_tooth_91_per_case': {'total_nb_tooth_91_per_day': 'sum',
                           'avg_nb_tooth_91_per_case': 'mean'},
    'nb_tooth_92_per_case': {'total_nb_tooth_92_per_day': 'sum',
                           'avg_nb_tooth_92_per_case': 'mean'}
}

In [51]:
# Group data_set by 'lab_name', 'month', 'day'
g_by_day = case_groups.groupby(by=['lab_name', 'month', 'day'])
day_groups = g_by_day.agg(agg_by_day)
day_groups.head()

subscriber_plan_name  \
                                        subscriber_plan_name   
lab_name             month   day                               
3DDENTALLABORATORIES 2017-06 2017-06-28             Standard   
                             2017-06-29             Standard   
                             2017-06-30             Standard   
                     2017-07 2017-07-03             Standard   
                             2017-07-05             Standard   

                                        nb_subscriber_features  \
                                        nb_subscriber_features   
lab_name             month   day                                 
3DDENTALLABORATORIES 2017-06 2017-06-28                     16   
                             2017-06-29                     16   
                             2017-06-30                     16   
                     2017-07 2017-07-03                     16   
                             2017-07-05                     16   

                                                 case_id         nb_units  \
                                        nb_cases_per_day nb_units_per_day   
lab_name             month   day                                            
3DDENTALLABORATORIES 2017-06 2017-06-28                1                3   
                             2017-06-29                2                4   
                             2017-06-30               12               17   
                     2017-07 2017-07-03                8                9   
                             2017-07-05                4                4   

                                                               \
                                        avg_nb_units_per_case   
lab_name             month   day                                
3DDENTALLABORATORIES 2017-06 2017-06-28              3.000000   
                             2017-06-29              2.000000   
                             2017-06-30              1.416667   
                     2017-07 2017-07-03              1.125000   
                             2017-07-05              1.000000   

                                        nb_products_per_case  \
                                         nb_products_per_day   
lab_name             month   day                               
3DDENTALLABORATORIES 2017-06 2017-06-28                    3   
                             2017-06-29                    4   
                             2017-06-30                   17   
                     2017-07 2017-07-03                    9   
                             2017-07-05                    4   

                                                                  \
                                        avg_nb_products_per_case   
lab_name             month   day                                   
3DDENTALLABORATORIES 2017-06 2017-06-28                 3.000000   
                             2017-06-29                 2.000000   
                             2017-06-30                 1.416667   
                     2017-07 2017-07-03                 1.125000   
                             2017-07-05                 1.000000   

                                        avg_nb_products_per_unit  \
                                        avg_nb_products_per_unit   
lab_name             month   day                                   
3DDENTALLABORATORIES 2017-06 2017-06-28                      1.0   
                             2017-06-29                      1.0   
                             2017-06-30                      1.0   
                     2017-07 2017-07-03                      1.0   
                             2017-07-05                      1.0   

                                        nb_product_types_per_case  \
                                         nb_product_types_per_day   
lab_name             month   day                                    
3DDENTALLABORATORIES 2017-06 2017-06-28           

In [52]:
day_groups.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 289002 entries, (3DDENTALLABORATORIES, 2017-06, 2017-06-28) to (ZAHNMACHER, 2019-05, 2019-05-03)
Data columns (total 91 columns):
(subscriber_plan_name, subscriber_plan_name)                        289002 non-null object
(nb_subscriber_features, nb_subscriber_features)                    289002 non-null int64
(case_id, nb_cases_per_day)                                         289002 non-null int64
(nb_units, nb_units_per_day)                                        289002 non-null int64
(nb_units, avg_nb_units_per_case)                                   289002 non-null float64
(nb_products_per_case, nb_products_per_day)                         289002 non-null int64
(nb_products_per_case, avg_nb_products_per_case)                    289002 non-null float64
(avg_nb_products_per_unit, avg_nb_products_per_unit)                289002 non-null float64
(nb_product_types_per_case, nb_product_types_per_day)               289002 non-null int64
(nb

In [53]:
day_groups.columns = day_groups.columns.droplevel(level=0)
day_groups.head()

subscriber_plan_name  \
lab_name             month   day                               
3DDENTALLABORATORIES 2017-06 2017-06-28             Standard   
                             2017-06-29             Standard   
                             2017-06-30             Standard   
                     2017-07 2017-07-03             Standard   
                             2017-07-05             Standard   

                                         nb_subscriber_features  \
lab_name             month   day                                  
3DDENTALLABORATORIES 2017-06 2017-06-28                      16   
                             2017-06-29                      16   
                             2017-06-30                      16   
                     2017-07 2017-07-03                      16   
                             2017-07-05                      16   

                                         nb_cases_per_day  nb_units_per_day  \
lab_name             month   day                                              
3DDENTALLABORATORIES 2017-06 2017-06-28                 1                 3   
                             2017-06-29                 2                 4   
                             2017-06-30                12                17   
                     2017-07 2017-07-03                 8                 9   
                             2017-07-05                 4                 4   

                                         avg_nb_units_per_case  \
lab_name             month   day                                 
3DDENTALLABORATORIES 2017-06 2017-06-28               3.000000   
                             2017-06-29               2.000000   
                             2017-06-30               1.416667   
                     2017-07 2017-07-03               1.125000   
                             2017-07-05               1.000000   

                                         nb_products_per_day  \
lab_name             month   day                               
3DDENTALLABORATORIES 2017-06 2017-06-28                    3   
                             2017-06-29                    4   
                             2017-06-30                   17   
                     2017-07 2017-07-03                    9   
                             2017-07-05                    4   

                                         avg_nb_products_per_case  \
lab_name             month   day                                    
3DDENTALLABORATORIES 2017-06 2017-06-28                  3.000000   
                             2017-06-29                  2.000000   
                             2017-06-30                  1.416667   
                     2017-07 2017-07-03                  1.125000   
                             2017-07-05                  1.000000   

                                         avg_nb_products_per_unit  \
lab_name             month   day                                    
3DDENTALLABORATORIES 2017-06 2017-06-28                       1.0   
                             2017-06-29                       1.0   
                             2017-06-30                       1.0   
                     2017-07 2017-07-03                       1.0   
                             2017-07-05                       1.0   

                                         nb_product_types_per_day  \
lab_name             month   day                                    
3DDENTALLABORATORIES 2017-06 2017-06-28                         3   
                             2017-06-29                         4   
                             2017-06-30                        17   
                     2017-07 2017-07-03                         9   
                             2017-07-05                         4   

                                         avg_nb_product_types_per_case  \
lab_name             month   day                                         
3DDENTALLABORATORIES 2017-06 2017-06-28                

In [54]:
day_groups.reset_index(inplace=True)
day_groups.head()

,lab_name,month,day,subscriber_plan_name,nb_subscriber_features,nb_cases_per_day,nb_units_per_day,avg_nb_units_per_case,nb_products_per_day,avg_nb_products_per_case,...,total_nb_tooth_31_per_day,avg_nb_tooth_31_per_case,total_nb_tooth_32_per_day,avg_nb_tooth_32_per_case,total_nb_tooth_90_per_day,avg_nb_tooth_90_per_case,total_nb_tooth_91_per_day,avg_nb_tooth_91_per_case,total_nb_tooth_92_per_day,avg_nb_tooth_92_per_case
0,3DDENTALLABORATORIES,2017-06,2017-06-28,Standard,16,1,3,3.000000,3,3.000000,...,0,0.000000,0,0.0,0.0,0.0,0,0.0,0,0.0
1,3DDENTALLABORATORIES,2017-06,2017-06-29,Standard,16,2,4,2.000000,4,2.000000,...,0,0.000000,0,0.0,0.0,0.0,0,0.0,0,0.0
2,3DDENTALLABORATORIES,2017-06,2017-06-30,Standard,16,12,17,1.416667,17,1.416667,...,2,0.166667,0,0.0,0.0,0.0,0,0.0,0,0.0
3,3DDENTALLABORATORIES,2017-07,2017-07-03,Standard,16,8,9,1.125000,9,1.125000,...,0,0.000000,0,0.0,0.0,0.0,0,0.0,0,0.0
4,3DDENTALLABORATORIES,2017-07,2017-07-05,Standard,16,4,4,1.000000,4,1.000000,...,0,0.000000,0,0.0,0.0,0.0,0,0.0,0,0.0


# FILE FOR A POTENTIAL TIME SERIES ANALYSIS!!!

In [55]:
# THIS FILE IS THE TIME SERIES TO USE FOR FORECASTING!!!
# Save to csv
day_groups.to_csv('data/file_for_analysis_daily_ALL.csv', index=False, header=True)

# YAY!

In [ ]:
day_groups = pd.read_csv('data/file_for_analysis_daily_ALL.csv')

In [56]:
day_groups.columns

Index(['lab_name', 'month', 'day', 'subscriber_plan_name',
       'nb_subscriber_features', 'nb_cases_per_day', 'nb_units_per_day',
       'avg_nb_units_per_case', 'nb_products_per_day',
       'avg_nb_products_per_case', 'avg_nb_products_per_unit',
       'nb_product_types_per_day', 'avg_nb_product_types_per_case',
       'avg_nb_product_types_per_unit', 'total_nb_other_settings_per_day',
       'avg_nb_other_settings_per_case', 'avg_nb_other_settings_per_unit',
       'nb_other_products_per_day', 'avg_nb_other_products_per_case',
       'nb_clients_per_day', 'nb_shipping_zipcodes_per_day', 'days_subscribed',
       'total_nb_tooth_0_per_day', 'avg_nb_tooth_0_per_case',
       'total_nb_tooth_1_per_day', 'avg_nb_tooth_1_per_case',
       'total_nb_tooth_2_per_day', 'avg_nb_tooth_2_per_case',
       'total_nb_tooth_3_per_day', 'avg_nb_tooth_3_per_case',
       'total_nb_tooth_4_per_day', 'avg_nb_tooth_4_per_case',
       'total_nb_tooth_5_per_day', 'avg_nb_tooth_5_per_case',
       'to

In [57]:
# Set aggregations for groupby by 'lab_name', 'month', 'day'
agg_by_month = {
    'subscriber_plan_name': {'subscriber_plan_name': 'first'},
    'nb_subscriber_features': {'nb_subscriber_features': 'first'},
    'nb_cases_per_day': {'total_nb_cases_per_month': 'sum',
                        'avg_nb_cases_per_day': 'mean'},
    'nb_units_per_day': {'total_nb_units_per_month': 'sum',
                        'avg_nb_units_per_day': 'mean'},
    'avg_nb_units_per_case': {'avg_nb_units_per_case': 'mean'},
    'nb_products_per_day': {'total_nb_products_per_month': 'sum',
                           'avg_nb_products_per_day': 'mean'},
    'avg_nb_products_per_case': {'avg_nb_products_per_case': 'mean'},
    'avg_nb_products_per_unit': {'avg_nb_products_per_unit': 'mean'},
    'nb_product_types_per_day': {'total_nb_product_types_per_month': 'sum',
                                'avg_nb_product_types_per_day': 'mean'},
    'avg_nb_product_types_per_case': {'avg_nb_product_types_per_case': 'mean'},   
    'avg_nb_product_types_per_unit': {'avg_nb_product_types_per_unit': 'mean'},
    'total_nb_other_settings_per_day': {'total_nb_other_settings_per_month': 'sum',
                                       'avg_nb_other_settings_per_day': 'mean'},
    'avg_nb_other_settings_per_case': {'avg_nb_other_settings_per_case': 'mean'},
    'avg_nb_other_settings_per_unit': {'avg_nb_other_settings_per_unit': 'mean'},
    'nb_other_products_per_day': {'total_nb_other_products_per_month': 'sum',
                                 'avg_nb_other_products_per_day': 'mean'},
    'avg_nb_other_products_per_case': {'avg_nb_other_products_per_case': 'mean'},
    'nb_clients_per_day': {'total_nb_clients_per_month': 'sum',
                          'avg_nb_clients_per_day': 'mean'},
    'nb_shipping_zipcodes_per_day': {'total_nb_shipping_zipcodes_per_month': 'sum',
                                    'avg_nb_shipping_zipcodes_per_day': 'mean'},   
    'days_subscribed': {'days_subscribed': 'first'},
    'total_nb_tooth_0_per_day': {'total_nb_tooth_0_per_month': 'sum',
                                 'avg_nb_tooth_0_per_day': 'mean'},
    'avg_nb_tooth_0_per_case': {'avg_nb_tooth_0_per_case': 'mean'},
    'total_nb_tooth_1_per_day': {'total_nb_tooth_1_per_month': 'sum',
                                 'avg_nb_tooth_1_per_day': 'mean'},
    'avg_nb_tooth_1_per_case': {'avg_nb_tooth_1_per_case': 'mean'},
    'total_nb_tooth_2_per_day': {'total_nb_tooth_2_per_month': 'sum',
                                 'avg_nb_tooth_2_per_day': 'mean'},
    'avg_nb_tooth_2_per_case': {'avg_nb_tooth_2_per_case': 'mean'},
    'total_nb_tooth_3_per_day': {'total_nb_tooth_3_per_month': 'sum',
                                 'avg_nb_tooth_3_per_day': 'mean'},
    'avg_nb_tooth_3_per_case': {'avg_nb_tooth_3_per_case': 'mean'},
    'total_nb_tooth_4_per_day': {'total_nb_tooth_4_per_month': 'sum',
                                 'avg_nb_tooth_4_per_day': 'mean'},
    'avg_nb_tooth_4_per_case': {'avg_nb_tooth_4_per_case': 'mean'},
    'total_nb_tooth_5_per_day': {'total_nb_tooth_5_per_month': 'sum',
                                 'avg_nb_tooth_5_per_day': 'mean'},
    'avg_nb_tooth_5_per_case': {'avg_nb_tooth_5_per_case': 'mean'},
    'total_nb_tooth_6_per_day': {'total_nb_tooth_6_per_month': 'sum',
                                 'avg_nb_tooth_6_per_day': 'mean'},
    'avg_nb_tooth_6_per_case': {'avg_nb_tooth_6_per_case': 'mean'},
    'total_nb_tooth_7_per_day': {'total_nb_tooth_7_per_month': 'sum',
                                 'avg_nb_tooth_7_per_day': 'mean'},
    'avg_nb_tooth_7_per_case': {'avg_nb_tooth_7_per_case': 'mean'},
    'total_nb_tooth_8_per_day': {'total_nb_tooth_8_per_month': 'sum',
                                 'avg_nb_tooth_8_per_day': 'mean'},
    'avg_nb_tooth_8_per_case': {'avg_nb_tooth_8_per_case': 'mean'},
    'total_nb_tooth_9_per_day': {'total_nb_tooth_9_per_month': 'sum',
                                 'avg_nb_tooth_9_per_day': 'mean'},
    'avg_nb_tooth_9_per_case': {'avg_nb_tooth_9_per_case': 'mean'},
    'total_nb_tooth_10_per_day': {'total_nb_tooth_10_per_month': 'sum',
                                 'avg_nb_tooth_10_per_day': 'mean'},
    'avg_nb_tooth_10_per_case': {'avg_nb_tooth_10_per_case': 'mean'},
    'total_nb_tooth_11_per_day': {'total_nb_tooth_11_per_month': 'sum',
                                 'avg_nb_tooth_11_per_day': 'mean'},
    'avg_nb_tooth_11_per_case': {'avg_nb_tooth_11_per_case': 'mean'},
    'total_nb_tooth_12_per_day': {'total_nb_tooth_12_per_month': 'sum',
                                 'avg_nb_tooth_12_per_day': 'mean'},
    'avg_nb_tooth_12_per_case': {'avg_nb_tooth_12_per_case': 'mean'},
    'total_nb_tooth_13_per_day': {'total_nb_tooth_13_per_month': 'sum',
                                 'avg_nb_tooth_13_per_day': 'mean'},
    'avg_nb_tooth_13_per_case': {'avg_nb_tooth_13_per_case': 'mean'},
    'total_nb_tooth_14_per_day': {'total_nb_tooth_14_per_month': 'sum',
                                 'avg_nb_tooth_14_per_day': 'mean'},
    'avg_nb_tooth_14_per_case': {'avg_nb_tooth_14_per_case': 'mean'},
    'total_nb_tooth_15_per_day': {'total_nb_tooth_15_per_month': 'sum',
                                 'avg_nb_tooth_15_per_day': 'mean'},
    'avg_nb_tooth_15_per_case': {'avg_nb_tooth_15_per_case': 'mean'},
    'total_nb_tooth_16_per_day': {'total_nb_tooth_16_per_month': 'sum',
                                 'avg_nb_tooth_16_per_day': 'mean'},
    'avg_nb_tooth_16_per_case': {'avg_nb_tooth_16_per_case': 'mean'},
    'total_nb_tooth_17_per_day': {'total_nb_tooth_17_per_month': 'sum',
                                 'avg_nb_tooth_17_per_day': 'mean'},
    'avg_nb_tooth_17_per_case': {'avg_nb_tooth_17_per_case': 'mean'},
    'total_nb_tooth_18_per_day': {'total_nb_tooth_18_per_month': 'sum',
                                 'avg_nb_tooth_18_per_day': 'mean'},
    'avg_nb_tooth_18_per_case': {'avg_nb_tooth_18_per_case': 'mean'},
    'total_nb_tooth_19_per_day': {'total_nb_tooth_19_per_month': 'sum',
                                 'avg_nb_tooth_19_per_day': 'mean'},
    'avg_nb_tooth_19_per_case': {'avg_nb_tooth_19_per_case': 'mean'},
    'total_nb_tooth_20_per_day': {'total_nb_tooth_20_per_month': 'sum',
                                 'avg_nb_tooth_20_per_day': 'mean'},
    'avg_nb_tooth_20_per_case': {'avg_nb_tooth_20_per_case': 'mean'},
    'total_nb_tooth_21_per_day': {'total_nb_tooth_21_per_month': 'sum',
                                 'avg_nb_tooth_21_per_day': 'mean'},
    'avg_nb_tooth_21_per_case': {'avg_nb_tooth_21_per_case': 'mean'},
    'total_nb_tooth_22_per_day': {'total_nb_tooth_22_per_month': 'sum',
                                 'avg_nb_tooth_22_per_day': 'mean'},
    'avg_nb_tooth_22_per_case': {'avg_nb_tooth_22_per_case': 'mean'},
    'total_nb_tooth_23_per_day': {'total_nb_tooth_23_per_month': 'sum',
                                 'avg_nb_tooth_23_per_day': 'mean'},
    'avg_nb_tooth_23_per_case': {'avg_nb_tooth_23_per_case': 'mean'},
    'total_nb_tooth_24_per_day': {'total_nb_tooth_24_per_month': 'sum',
                                 'avg_nb_tooth_24_per_day': 'mean'},
    'avg_nb_tooth_24_per_case': {'avg_nb_tooth_24_per_case': 'mean'},
    'total_nb_tooth_25_per_day': {'total_nb_tooth_25_per_month': 'sum',
                                 'avg_nb_tooth_25_per_day': 'mean'},
    'avg_nb_tooth_25_per_case': {'avg_nb_tooth_25_per_case': 'mean'},
    'total_nb_tooth_26_per_day': {'total_nb_tooth_26_per_month': 'sum',
                                 'avg_nb_tooth_26_per_day': 'mean'},
    'avg_nb_tooth_26_per_case': {'avg_nb_tooth_26_per_case': 'mean'},
    'total_nb_tooth_27_per_day': {'total_nb_tooth_27_per_month': 'sum',
                                 'avg_nb_tooth_27_per_day': 'mean'},
    'avg_nb_tooth_27_per_case': {'avg_nb_tooth_27_per_case': 'mean'},
    'total_nb_tooth_28_per_day': {'total_nb_tooth_28_per_month': 'sum',
                                 'avg_nb_tooth_28_per_day': 'mean'},
    'avg_nb_tooth_28_per_case': {'avg_nb_tooth_28_per_case': 'mean'},
    'total_nb_tooth_29_per_day': {'total_nb_tooth_29_per_month': 'sum',
                                 'avg_nb_tooth_29_per_day': 'mean'},
    'avg_nb_tooth_29_per_case': {'avg_nb_tooth_29_per_case': 'mean'},
    'total_nb_tooth_30_per_day': {'total_nb_tooth_30_per_month': 'sum',
                                 'avg_nb_tooth_30_per_day': 'mean'},
    'avg_nb_tooth_30_per_case': {'avg_nb_tooth_30_per_case': 'mean'},
    'total_nb_tooth_31_per_day': {'total_nb_tooth_31_per_month': 'sum',
                                 'avg_nb_tooth_31_per_day': 'mean'},
    'avg_nb_tooth_31_per_case': {'avg_nb_tooth_31_per_case': 'mean'},
    'total_nb_tooth_32_per_day': {'total_nb_tooth_32_per_month': 'sum',
                                 'avg_nb_tooth_32_per_day': 'mean'},
    'avg_nb_tooth_32_per_case': {'avg_nb_tooth_32_per_case': 'mean'},
    'total_nb_tooth_90_per_day': {'total_nb_tooth_90_per_month': 'sum',
                                 'avg_nb_tooth_90_per_day': 'mean'},
    'avg_nb_tooth_90_per_case': {'avg_nb_tooth_90_per_case': 'mean'},
    'total_nb_tooth_91_per_day': {'total_nb_tooth_91_per_month': 'sum',
                                 'avg_nb_tooth_91_per_day': 'mean'},
    'avg_nb_tooth_91_per_case': {'avg_nb_tooth_91_per_case': 'mean'},
    'total_nb_tooth_92_per_day': {'total_nb_tooth_92_per_month': 'sum',
                                 'avg_nb_tooth_92_per_day': 'mean'},
    'avg_nb_tooth_92_per_case': {'avg_nb_tooth_92_per_case': 'mean'}
}

In [58]:
# Group data_set by 'lab_name', 'month'
g_by_month = day_groups.groupby(by=['lab_name', 'month'])
month_groups = g_by_month.agg(agg_by_month)
month_groups.head()

subscriber_plan_name nb_subscriber_features  \
                             subscriber_plan_name nb_subscriber_features   
lab_name             month                                                 
3DDENTALLABORATORIES 2017-06             Standard                     16   
                     2017-07             Standard                     16   
                     2017-08             Standard                     16   
                     2017-09             Standard                     16   
                     2017-10             Standard                     16   

                                     nb_cases_per_day                       \
                             total_nb_cases_per_month avg_nb_cases_per_day   
lab_name             month                                                   
3DDENTALLABORATORIES 2017-06                       15             5.000000   
                     2017-07                      146             6.952381   
                     2017-08                      149             6.772727   
                     2017-09                      105             5.833333   
                     2017-10                      143             6.500000   

                                     nb_units_per_day                       \
                             total_nb_units_per_month avg_nb_units_per_day   
lab_name             month                                                   
3DDENTALLABORATORIES 2017-06                       24             8.000000   
                     2017-07                      214            10.190476   
                     2017-08                      213             9.681818   
                     2017-09                      154             8.555556   
                     2017-10                      195             8.863636   

                             avg_nb_units_per_case  \
                             avg_nb_units_per_case   
lab_name             month                           
3DDENTALLABORATORIES 2017-06              2.138889   
                     2017-07              1.453798   
                     2017-08              1.385342   
                     2017-09              1.510061   
                     2017-10              1.296824   

                                     nb_products_per_day  \
                             total_nb_products_per_month   
lab_name             month                                 
3DDENTALLABORATORIES 2017-06                          24   
                     2017-07                         214   
                     2017-08                         213   
                     2017-09                         154   
                     2017-10                         195   

                                                     avg_nb_products_per_case  \
                             avg_nb_products_per_day avg_nb_products_per_case   
lab_name             month                                                      
3DDENTALLABORATORIES 2017-06                8.000000                 2.138889   
                     2017-07               10.190476                 1.453798   
                     2017-08                9.681818                 1.385342   
                     2017-09                8.555556                 1.510061   
                     2017-10                8.863636                 1.296824   

                                       ...             \
                                       ...              
lab_name             month             ...              
3DDENTALLABORATORIES 2017-06           ...              
                     2017-07           ...              
                     2017-08           ...              
                     2017-09           ...              
                     2017-10           ...              

                             avg_nb_tooth_32_per_case  \
                             avg_nb_tooth_32_per_case   
lab_name             month          

In [59]:
month_groups.columns = month_groups.columns.droplevel(level=0)
month_groups.head()

subscriber_plan_name  nb_subscriber_features  \
lab_name             month                                                  
3DDENTALLABORATORIES 2017-06             Standard                      16   
                     2017-07             Standard                      16   
                     2017-08             Standard                      16   
                     2017-09             Standard                      16   
                     2017-10             Standard                      16   

                              total_nb_cases_per_month  avg_nb_cases_per_day  \
lab_name             month                                                     
3DDENTALLABORATORIES 2017-06                        15              5.000000   
                     2017-07                       146              6.952381   
                     2017-08                       149              6.772727   
                     2017-09                       105              5.833333   
                     2017-10                       143              6.500000   

                              total_nb_units_per_month  avg_nb_units_per_day  \
lab_name             month                                                     
3DDENTALLABORATORIES 2017-06                        24              8.000000   
                     2017-07                       214             10.190476   
                     2017-08                       213              9.681818   
                     2017-09                       154              8.555556   
                     2017-10                       195              8.863636   

                              avg_nb_units_per_case  \
lab_name             month                            
3DDENTALLABORATORIES 2017-06               2.138889   
                     2017-07               1.453798   
                     2017-08               1.385342   
                     2017-09               1.510061   
                     2017-10               1.296824   

                              total_nb_products_per_month  \
lab_name             month                                  
3DDENTALLABORATORIES 2017-06                           24   
                     2017-07                          214   
                     2017-08                          213   
                     2017-09                          154   
                     2017-10                          195   

                              avg_nb_products_per_day  \
lab_name             month                              
3DDENTALLABORATORIES 2017-06                 8.000000   
                     2017-07                10.190476   
                     2017-08                 9.681818   
                     2017-09                 8.555556   
                     2017-10                 8.863636   

                              avg_nb_products_per_case  \
lab_name             month                               
3DDENTALLABORATORIES 2017-06                  2.138889   
                     2017-07                  1.453798   
                     2017-08                  1.385342   
                     2017-09                  1.510061   
                     2017-10                  1.296824   

                                        ...             \
lab_name             month              ...              
3DDENTALLABORATORIES 2017-06            ...              
                     2017-07            ...              
                     2017-08            ...              
                     2017-09            ...              
                     2017-10            ...              

                              avg_nb_tooth_32_per_case  \
lab_name             month                               
3DDENTALLABORATORIES 2017-06                  0.000000   
                     2017-07                  0.009524   
                     2017-08                  0.000000   
                     2017-09                  0.0

In [60]:
month_groups.reset_index(inplace=True)
month_groups.head()

,lab_name,month,subscriber_plan_name,nb_subscriber_features,total_nb_cases_per_month,avg_nb_cases_per_day,total_nb_units_per_month,avg_nb_units_per_day,avg_nb_units_per_case,total_nb_products_per_month,...,avg_nb_tooth_32_per_case,total_nb_tooth_90_per_month,avg_nb_tooth_90_per_day,avg_nb_tooth_90_per_case,total_nb_tooth_91_per_month,avg_nb_tooth_91_per_day,avg_nb_tooth_91_per_case,total_nb_tooth_92_per_month,avg_nb_tooth_92_per_day,avg_nb_tooth_92_per_case
0,3DDENTALLABORATORIES,2017-06,Standard,16,15,5.000000,24,8.000000,2.138889,24,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3DDENTALLABORATORIES,2017-07,Standard,16,146,6.952381,214,10.190476,1.453798,214,...,0.009524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3DDENTALLABORATORIES,2017-08,Standard,16,149,6.772727,213,9.681818,1.385342,213,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3DDENTALLABORATORIES,2017-09,Standard,16,105,5.833333,154,8.555556,1.510061,154,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3DDENTALLABORATORIES,2017-10,Standard,16,143,6.500000,195,8.863636,1.296824,195,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
month_groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16090 entries, 0 to 16089
Columns: 137 entries, lab_name to avg_nb_tooth_92_per_case
dtypes: float64(115), int64(8), object(3), uint8(11)
memory usage: 15.6+ MB


# SAVE MONTHLY AGGREGATION TO CSV!!

In [62]:
# THIS FILE IS THE TIME SERIES TO USE FOR FORECASTING!!!
# Save to csv
month_groups.to_csv('data/file_for_analysis_monthly_ALL.csv', index=False, header=True)

# YAY

In [63]:
month_groups.columns

Index(['lab_name', 'month', 'subscriber_plan_name', 'nb_subscriber_features',
       'total_nb_cases_per_month', 'avg_nb_cases_per_day',
       'total_nb_units_per_month', 'avg_nb_units_per_day',
       'avg_nb_units_per_case', 'total_nb_products_per_month',
       ...
       'avg_nb_tooth_32_per_case', 'total_nb_tooth_90_per_month',
       'avg_nb_tooth_90_per_day', 'avg_nb_tooth_90_per_case',
       'total_nb_tooth_91_per_month', 'avg_nb_tooth_91_per_day',
       'avg_nb_tooth_91_per_case', 'total_nb_tooth_92_per_month',
       'avg_nb_tooth_92_per_day', 'avg_nb_tooth_92_per_case'],
      dtype='object', length=137)

In [64]:
# Set aggregations for groupby by 'lab_name'
agg_by_lab = {
    'subscriber_plan_name': {'subscriber_plan_name': 'first'},
    'nb_subscriber_features': {'nb_subscriber_features': 'first'},
    'total_nb_cases_per_month': {'total_nb_cases': 'sum',
                                'avg_nb_cases_per_month': 'mean'},
    'avg_nb_cases_per_day': {'avg_nb_cases_per_day': 'mean'},
    'total_nb_units_per_month': {'total_nb_units': 'sum',
                                'avg_nb_units_per_month': 'mean'},
    'avg_nb_units_per_day': {'avg_nb_units_per_day': 'mean'},
    'avg_nb_units_per_case': {'avg_nb_units_per_case': 'mean'},
    'total_nb_products_per_month': {'total_nb_products': 'sum',
                                'avg_nb_products_per_month': 'mean'},
    'avg_nb_products_per_day': {'avg_nb_products_per_day': 'mean'},
    'avg_nb_products_per_case': {'avg_nb_products_per_case': 'mean'},   
    'avg_nb_products_per_unit': {'avg_nb_products_per_unit': 'mean'},
    'total_nb_product_types_per_month': {'total_nb_product_types': 'sum',
                                'avg_nb_product_types_per_month': 'mean'},
    'avg_nb_product_types_per_day': {'avg_nb_product_types_per_day': 'mean'},
    'avg_nb_product_types_per_case': {'avg_nb_product_types_per_case': 'mean'},   
    'avg_nb_product_types_per_unit': {'avg_nb_product_types_per_unit': 'mean'},
    'total_nb_other_settings_per_month': {'total_nb_other_settings': 'sum',
                                'avg_nb_other_settings_per_month': 'mean'},
    'avg_nb_other_settings_per_day': {'avg_nb_other_settings_per_day': 'mean'},
    'avg_nb_other_settings_per_case': {'avg_nb_other_settings_per_case': 'mean'},
    'avg_nb_other_settings_per_unit': {'avg_nb_other_settings_per_unit': 'mean'},
    'total_nb_other_products_per_month': {'total_nb_other_products': 'sum',
                                'avg_nb_other_products_per_month': 'mean'},
    'avg_nb_other_products_per_day': {'avg_nb_other_products_per_day': 'mean'},
    'avg_nb_other_products_per_case': {'avg_nb_other_products_per_case': 'mean'},
    'total_nb_clients_per_month': {'total_nb_clients': 'sum',
                                  'avg_nb_clients_per_month': 'mean'},
    'avg_nb_clients_per_day': {'avg_nb_clients_per_day': 'mean'},
    'total_nb_shipping_zipcodes_per_month': {'total_nb_shipping_zipcodes': 'sum',
                                  'avg_nb_shipping_zipcodes_per_month': 'mean'},
    'avg_nb_shipping_zipcodes_per_day': {'avg_nb_shipping_zipcodes_per_day': 'mean'},  
    'days_subscribed': {'days_subscribed': 'first'},
    'total_nb_tooth_0_per_month': {'total_nb_tooth_0': 'sum',
                                  'avg_nb_tooth_0_per_month': 'mean'},
    'avg_nb_tooth_0_per_day': {'avg_nb_tooth_0_per_day': 'mean'},
    'avg_nb_tooth_0_per_case': {'avg_nb_tooth_0_per_case': 'mean'},
    'total_nb_tooth_1_per_month': {'total_nb_tooth_1': 'sum',
                                  'avg_nb_tooth_1_per_month': 'mean'},
    'avg_nb_tooth_1_per_day': {'avg_nb_tooth_1_per_day': 'mean'},
    'avg_nb_tooth_1_per_case': {'avg_nb_tooth_1_per_case': 'mean'},
    'total_nb_tooth_2_per_month': {'total_nb_tooth_2': 'sum',
                                  'avg_nb_tooth_2_per_month': 'mean'},
    'avg_nb_tooth_2_per_day': {'avg_nb_tooth_2_per_day': 'mean'},
    'avg_nb_tooth_2_per_case': {'avg_nb_tooth_2_per_case': 'mean'},
    'total_nb_tooth_3_per_month': {'total_nb_tooth_3': 'sum',
                                  'avg_nb_tooth_3_per_month': 'mean'},
    'avg_nb_tooth_3_per_day': {'avg_nb_tooth_3_per_day': 'mean'},
    'avg_nb_tooth_3_per_case': {'avg_nb_tooth_3_per_case': 'mean'},
    'total_nb_tooth_4_per_month': {'total_nb_tooth_4': 'sum',
                                  'avg_nb_tooth_4_per_month': 'mean'},
    'avg_nb_tooth_4_per_day': {'avg_nb_tooth_4_per_day': 'mean'},
    'avg_nb_tooth_4_per_case': {'avg_nb_tooth_4_per_case': 'mean'},
    'total_nb_tooth_5_per_month': {'total_nb_tooth_5': 'sum',
                                  'avg_nb_tooth_5_per_month': 'mean'},
    'avg_nb_tooth_5_per_day': {'avg_nb_tooth_5_per_day': 'mean'},
    'avg_nb_tooth_5_per_case': {'avg_nb_tooth_5_per_case': 'mean'},
    'total_nb_tooth_6_per_month': {'total_nb_tooth_6': 'sum',
                                  'avg_nb_tooth_6_per_month': 'mean'},
    'avg_nb_tooth_6_per_day': {'avg_nb_tooth_6_per_day': 'mean'},
    'avg_nb_tooth_6_per_case': {'avg_nb_tooth_6_per_case': 'mean'},
    'total_nb_tooth_7_per_month': {'total_nb_tooth_7': 'sum',
                                  'avg_nb_tooth_7_per_month': 'mean'},
    'avg_nb_tooth_7_per_day': {'avg_nb_tooth_7_per_day': 'mean'},
    'avg_nb_tooth_7_per_case': {'avg_nb_tooth_7_per_case': 'mean'},
    'total_nb_tooth_8_per_month': {'total_nb_tooth_8': 'sum',
                                  'avg_nb_tooth_8_per_month': 'mean'},
    'avg_nb_tooth_8_per_day': {'avg_nb_tooth_8_per_day': 'mean'},
    'avg_nb_tooth_8_per_case': {'avg_nb_tooth_8_per_case': 'mean'},
    'total_nb_tooth_9_per_month': {'total_nb_tooth_9': 'sum',
                                  'avg_nb_tooth_9_per_month': 'mean'},
    'avg_nb_tooth_9_per_day': {'avg_nb_tooth_9_per_day': 'mean'},
    'avg_nb_tooth_9_per_case': {'avg_nb_tooth_9_per_case': 'mean'},
    'total_nb_tooth_10_per_month': {'total_nb_tooth_10': 'sum',
                                  'avg_nb_tooth_10_per_month': 'mean'},
    'avg_nb_tooth_10_per_day': {'avg_nb_tooth_10_per_day': 'mean'},
    'avg_nb_tooth_10_per_case': {'avg_nb_tooth_10_per_case': 'mean'},
    'total_nb_tooth_11_per_month': {'total_nb_tooth_11': 'sum',
                                  'avg_nb_tooth_11_per_month': 'mean'},
    'avg_nb_tooth_11_per_day': {'avg_nb_tooth_11_per_day': 'mean'},
    'avg_nb_tooth_11_per_case': {'avg_nb_tooth_11_per_case': 'mean'},
    'total_nb_tooth_12_per_month': {'total_nb_tooth_12': 'sum',
                                  'avg_nb_tooth_12_per_month': 'mean'},
    'avg_nb_tooth_12_per_day': {'avg_nb_tooth_12_per_day': 'mean'},
    'avg_nb_tooth_12_per_case': {'avg_nb_tooth_12_per_case': 'mean'},
    'total_nb_tooth_13_per_month': {'total_nb_tooth_13': 'sum',
                                  'avg_nb_tooth_13_per_month': 'mean'},
    'avg_nb_tooth_13_per_day': {'avg_nb_tooth_13_per_day': 'mean'},
    'avg_nb_tooth_13_per_case': {'avg_nb_tooth_13_per_case': 'mean'},
    'total_nb_tooth_14_per_month': {'total_nb_tooth_14': 'sum',
                                  'avg_nb_tooth_14_per_month': 'mean'},
    'avg_nb_tooth_14_per_day': {'avg_nb_tooth_14_per_day': 'mean'},
    'avg_nb_tooth_14_per_case': {'avg_nb_tooth_14_per_case': 'mean'},
    'total_nb_tooth_15_per_month': {'total_nb_tooth_15': 'sum',
                                  'avg_nb_tooth_15_per_month': 'mean'},
    'avg_nb_tooth_15_per_day': {'avg_nb_tooth_15_per_day': 'mean'},
    'avg_nb_tooth_15_per_case': {'avg_nb_tooth_15_per_case': 'mean'},
    'total_nb_tooth_16_per_month': {'total_nb_tooth_16': 'sum',
                                  'avg_nb_tooth_16_per_month': 'mean'},
    'avg_nb_tooth_16_per_day': {'avg_nb_tooth_16_per_day': 'mean'},
    'avg_nb_tooth_16_per_case': {'avg_nb_tooth_16_per_case': 'mean'},
    'total_nb_tooth_17_per_month': {'total_nb_tooth_17': 'sum',
                                  'avg_nb_tooth_17_per_month': 'mean'},
    'avg_nb_tooth_17_per_day': {'avg_nb_tooth_17_per_day': 'mean'},
    'avg_nb_tooth_17_per_case': {'avg_nb_tooth_17_per_case': 'mean'},
    'total_nb_tooth_18_per_month': {'total_nb_tooth_18': 'sum',
                                  'avg_nb_tooth_18_per_month': 'mean'},
    'avg_nb_tooth_18_per_day': {'avg_nb_tooth_18_per_day': 'mean'},
    'avg_nb_tooth_18_per_case': {'avg_nb_tooth_18_per_case': 'mean'},
    'total_nb_tooth_19_per_month': {'total_nb_tooth_19': 'sum',
                                  'avg_nb_tooth_19_per_month': 'mean'},
    'avg_nb_tooth_19_per_day': {'avg_nb_tooth_19_per_day': 'mean'},
    'avg_nb_tooth_19_per_case': {'avg_nb_tooth_19_per_case': 'mean'},
    'total_nb_tooth_20_per_month': {'total_nb_tooth_20': 'sum',
                                  'avg_nb_tooth_20_per_month': 'mean'},
    'avg_nb_tooth_20_per_day': {'avg_nb_tooth_20_per_day': 'mean'},
    'avg_nb_tooth_20_per_case': {'avg_nb_tooth_20_per_case': 'mean'},
    'total_nb_tooth_21_per_month': {'total_nb_tooth_21': 'sum',
                                  'avg_nb_tooth_21_per_month': 'mean'},
    'avg_nb_tooth_21_per_day': {'avg_nb_tooth_21_per_day': 'mean'},
    'avg_nb_tooth_21_per_case': {'avg_nb_tooth_21_per_case': 'mean'},
    'total_nb_tooth_22_per_month': {'total_nb_tooth_22': 'sum',
                                  'avg_nb_tooth_22_per_month': 'mean'},
    'avg_nb_tooth_22_per_day': {'avg_nb_tooth_22_per_day': 'mean'},
    'avg_nb_tooth_22_per_case': {'avg_nb_tooth_22_per_case': 'mean'},
    'total_nb_tooth_23_per_month': {'total_nb_tooth_23': 'sum',
                                  'avg_nb_tooth_23_per_month': 'mean'},
    'avg_nb_tooth_23_per_day': {'avg_nb_tooth_23_per_day': 'mean'},
    'avg_nb_tooth_23_per_case': {'avg_nb_tooth_23_per_case': 'mean'},
    'total_nb_tooth_24_per_month': {'total_nb_tooth_24': 'sum',
                                  'avg_nb_tooth_24_per_month': 'mean'},
    'avg_nb_tooth_24_per_day': {'avg_nb_tooth_24_per_day': 'mean'},
    'avg_nb_tooth_24_per_case': {'avg_nb_tooth_24_per_case': 'mean'},
    'total_nb_tooth_25_per_month': {'total_nb_tooth_25': 'sum',
                                  'avg_nb_tooth_25_per_month': 'mean'},
    'avg_nb_tooth_25_per_day': {'avg_nb_tooth_25_per_day': 'mean'},
    'avg_nb_tooth_25_per_case': {'avg_nb_tooth_25_per_case': 'mean'},
    'total_nb_tooth_26_per_month': {'total_nb_tooth_26': 'sum',
                                  'avg_nb_tooth_26_per_month': 'mean'},
    'avg_nb_tooth_26_per_day': {'avg_nb_tooth_26_per_day': 'mean'},
    'avg_nb_tooth_26_per_case': {'avg_nb_tooth_26_per_case': 'mean'},
    'total_nb_tooth_27_per_month': {'total_nb_tooth_27': 'sum',
                                  'avg_nb_tooth_27_per_month': 'mean'},
    'avg_nb_tooth_27_per_day': {'avg_nb_tooth_27_per_day': 'mean'},
    'avg_nb_tooth_27_per_case': {'avg_nb_tooth_27_per_case': 'mean'},
    'total_nb_tooth_28_per_month': {'total_nb_tooth_28': 'sum',
                                  'avg_nb_tooth_28_per_month': 'mean'},
    'avg_nb_tooth_28_per_day': {'avg_nb_tooth_28_per_day': 'mean'},
    'avg_nb_tooth_28_per_case': {'avg_nb_tooth_28_per_case': 'mean'},
    'total_nb_tooth_29_per_month': {'total_nb_tooth_29': 'sum',
                                  'avg_nb_tooth_29_per_month': 'mean'},
    'avg_nb_tooth_29_per_day': {'avg_nb_tooth_29_per_day': 'mean'},
    'avg_nb_tooth_29_per_case': {'avg_nb_tooth_29_per_case': 'mean'},
    'total_nb_tooth_30_per_month': {'total_nb_tooth_30': 'sum',
                                  'avg_nb_tooth_30_per_month': 'mean'},
    'avg_nb_tooth_30_per_day': {'avg_nb_tooth_30_per_day': 'mean'},
    'avg_nb_tooth_30_per_case': {'avg_nb_tooth_30_per_case': 'mean'},
    'total_nb_tooth_31_per_month': {'total_nb_tooth_31': 'sum',
                                  'avg_nb_tooth_31_per_month': 'mean'},
    'avg_nb_tooth_31_per_day': {'avg_nb_tooth_31_per_day': 'mean'},
    'avg_nb_tooth_31_per_case': {'avg_nb_tooth_31_per_case': 'mean'},
    'total_nb_tooth_32_per_month': {'total_nb_tooth_32': 'sum',
                                  'avg_nb_tooth_32_per_month': 'mean'},
    'avg_nb_tooth_32_per_day': {'avg_nb_tooth_32_per_day': 'mean'},
    'avg_nb_tooth_32_per_case': {'avg_nb_tooth_32_per_case': 'mean'},
    'total_nb_tooth_90_per_month': {'total_nb_tooth_90': 'sum',
                                  'avg_nb_tooth_90_per_month': 'mean'},
    'avg_nb_tooth_90_per_day': {'avg_nb_tooth_90_per_day': 'mean'},
    'avg_nb_tooth_90_per_case': {'avg_nb_tooth_90_per_case': 'mean'},
    'total_nb_tooth_91_per_month': {'total_nb_tooth_91': 'sum',
                                  'avg_nb_tooth_91_per_month': 'mean'},
    'avg_nb_tooth_91_per_day': {'avg_nb_tooth_91_per_day': 'mean'},
    'avg_nb_tooth_91_per_case': {'avg_nb_tooth_91_per_case': 'mean'},
    'total_nb_tooth_92_per_month': {'total_nb_tooth_92': 'sum',
                                  'avg_nb_tooth_92_per_month': 'mean'},
    'avg_nb_tooth_92_per_day': {'avg_nb_tooth_92_per_day': 'mean'},
    'avg_nb_tooth_92_per_case': {'avg_nb_tooth_92_per_case': 'mean'}
}

In [65]:
# Group data_set by ab_name
g_by_lab = month_groups.groupby(by=['lab_name'], as_index=False)
lab_groups = g_by_lab.agg(agg_by_lab)
lab_groups.head()

lab_name subscriber_plan_name nb_subscriber_features  \
                        subscriber_plan_name nb_subscriber_features   
0  3DDENTALLABORATORIES             Standard                     16   
1               3DENTAL        StarterLegacy                     19   
2              3DLABSMT              Starter                      8   
3        3LLABORATORIES            Standard2                      8   
4                   AAA        StarterLegacy                     19   

  total_nb_cases_per_month                        avg_nb_cases_per_day  \
            total_nb_cases avg_nb_cases_per_month avg_nb_cases_per_day   
0                     2971             123.791667             6.463748   
1                     2343              34.455882             3.184887   
2                       64              21.333333             6.000000   
3                     1936             276.571429            17.630179   
4                    10949             243.311111            12.504408   

  total_nb_units_per_month                        avg_nb_units_per_day  \
            total_nb_units avg_nb_units_per_month avg_nb_units_per_day   
0                     4455             185.625000             9.641180   
1                     3433              50.485294             4.645158   
2                      113              37.666667            10.966667   
3                     2720             388.571429            24.741950   
4                    17471             388.244444            19.947611   

  avg_nb_units_per_case           ...            avg_nb_tooth_90_per_day  \
  avg_nb_units_per_case           ...            avg_nb_tooth_90_per_day   
0              1.507250           ...                           0.000000   
1              1.638398           ...                           0.689914   
2              1.800635           ...                           0.566667   
3              1.452787           ...                           0.513152   
4              1.743776           ...                           0.000000   

  avg_nb_tooth_90_per_case total_nb_tooth_91_per_month  \
  avg_nb_tooth_90_per_case           total_nb_tooth_91   
0                 0.000000                         0.0   
1                 0.173824                       345.0   
2                 0.103757                         2.0   
3                 0.070132                        33.0   
4                 0.000000                         0.0   

                            avg_nb_tooth_91_per_day avg_nb_tooth_91_per_case  \
  avg_nb_tooth_91_per_month avg_nb_tooth_91_per_day avg_nb_tooth_91_per_case   
0                  0.000000                0.000000                 0.000000   
1                  5.073529                0.471851                 0.123728   
2                  0.666667                0.200000                 0.028571   
3                  4.714286                0.319006                 0.070722   
4                  0.000000                0.000000                 0.000000   

  total_nb_tooth_92_per_month                            \
            total_nb_tooth_92 avg_nb_tooth_92_per_month   
0                         0.0                  0.000000   
1                       171.0                  2.514706   
2                        19.0                  6.333333   
3                        12.0                  1.714286   
4                         0.0                  0.000000   

  avg_nb_tooth_92_per_day avg_nb_tooth_92_per_case  
  avg_nb_tooth_92_per_day avg_nb_tooth_92_per_case  
0                0.000000                 0.000000  
1                0.252117                 0.056688  
2                1.700000                 0.249471  
3                0.079777                 0.008106  
4                0.000000                 0.000000  

[5 rows x 180 columns]

In [66]:
lab_groups.columns = lab_groups.columns.droplevel(level=0)
lab_groups.head()

,,subscriber_plan_name,nb_subscriber_features,total_nb_cases,avg_nb_cases_per_month,avg_nb_cases_per_day,total_nb_units,avg_nb_units_per_month,avg_nb_units_per_day,avg_nb_units_per_case,...,avg_nb_tooth_90_per_day,avg_nb_tooth_90_per_case,total_nb_tooth_91,avg_nb_tooth_91_per_month,avg_nb_tooth_91_per_day,avg_nb_tooth_91_per_case,total_nb_tooth_92,avg_nb_tooth_92_per_month,avg_nb_tooth_92_per_day,avg_nb_tooth_92_per_case
0,3DDENTALLABORATORIES,Standard,16,2971,123.791667,6.463748,4455,185.625000,9.641180,1.507250,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,3DENTAL,StarterLegacy,19,2343,34.455882,3.184887,3433,50.485294,4.645158,1.638398,...,0.689914,0.173824,345.0,5.073529,0.471851,0.123728,171.0,2.514706,0.252117,0.056688
2,3DLABSMT,Starter,8,64,21.333333,6.000000,113,37.666667,10.966667,1.800635,...,0.566667,0.103757,2.0,0.666667,0.200000,0.028571,19.0,6.333333,1.700000,0.249471
3,3LLABORATORIES,Standard2,8,1936,276.571429,17.630179,2720,388.571429,24.741950,1.452787,...,0.513152,0.070132,33.0,4.714286,0.319006,0.070722,12.0,1.714286,0.079777,0.008106
4,AAA,StarterLegacy,19,10949,243.311111,12.504408,17471,388.244444,19.947611,1.743776,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [67]:
lab_groups.rename(columns={'': 'lab_name'}, inplace=True)
lab_groups.columns

Index(['lab_name', 'subscriber_plan_name', 'nb_subscriber_features',
       'total_nb_cases', 'avg_nb_cases_per_month', 'avg_nb_cases_per_day',
       'total_nb_units', 'avg_nb_units_per_month', 'avg_nb_units_per_day',
       'avg_nb_units_per_case',
       ...
       'avg_nb_tooth_90_per_day', 'avg_nb_tooth_90_per_case',
       'total_nb_tooth_91', 'avg_nb_tooth_91_per_month',
       'avg_nb_tooth_91_per_day', 'avg_nb_tooth_91_per_case',
       'total_nb_tooth_92', 'avg_nb_tooth_92_per_month',
       'avg_nb_tooth_92_per_day', 'avg_nb_tooth_92_per_case'],
      dtype='object', length=180)

In [68]:
lab_groups.head()

,lab_name,subscriber_plan_name,nb_subscriber_features,total_nb_cases,avg_nb_cases_per_month,avg_nb_cases_per_day,total_nb_units,avg_nb_units_per_month,avg_nb_units_per_day,avg_nb_units_per_case,...,avg_nb_tooth_90_per_day,avg_nb_tooth_90_per_case,total_nb_tooth_91,avg_nb_tooth_91_per_month,avg_nb_tooth_91_per_day,avg_nb_tooth_91_per_case,total_nb_tooth_92,avg_nb_tooth_92_per_month,avg_nb_tooth_92_per_day,avg_nb_tooth_92_per_case
0,3DDENTALLABORATORIES,Standard,16,2971,123.791667,6.463748,4455,185.625000,9.641180,1.507250,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,3DENTAL,StarterLegacy,19,2343,34.455882,3.184887,3433,50.485294,4.645158,1.638398,...,0.689914,0.173824,345.0,5.073529,0.471851,0.123728,171.0,2.514706,0.252117,0.056688
2,3DLABSMT,Starter,8,64,21.333333,6.000000,113,37.666667,10.966667,1.800635,...,0.566667,0.103757,2.0,0.666667,0.200000,0.028571,19.0,6.333333,1.700000,0.249471
3,3LLABORATORIES,Standard2,8,1936,276.571429,17.630179,2720,388.571429,24.741950,1.452787,...,0.513152,0.070132,33.0,4.714286,0.319006,0.070722,12.0,1.714286,0.079777,0.008106
4,AAA,StarterLegacy,19,10949,243.311111,12.504408,17471,388.244444,19.947611,1.743776,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [69]:
lab_groups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471 entries, 0 to 470
Columns: 180 entries, lab_name to avg_nb_tooth_92_per_case
dtypes: float64(169), int64(8), object(2), uint8(1)
memory usage: 662.8+ KB


# FILE FOR THE CLUSTER ANALYSIS!!!

In [70]:
# THIS FILE IS TO BE USED FOR THE CLUSTER ANALYSIS (K means)!!!
# Save to csv
lab_groups.to_csv('data/df_cluster_analysis.csv', index=False, header=True)

# YOUPI!!!